In [1]:
# Load the "autoreload" extension so that code can change
%load_ext autoreload

# Always reload modules so that as you change code in src, it gets loaded
%autoreload 2


In [ ]:

pip install -r requirements.txt

In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:


from Auth.MsalClientAuthenticator import MsalClientAuthenticator

CLIENT_ID = os.environ.get('CLIENT_ID')
TENANT_ID = os.environ.get('TENANT_ID')
SECRET = os.environ.get('CLIENT_SECRET')
SCOPESC = [os.environ.get('SCOPE_DEFAULT')]

authenticator = MsalClientAuthenticator(CLIENT_ID, TENANT_ID, SECRET, SCOPESC)
try:
    token = authenticator.GetAuthtoken()
    print("Token acquired:", token)
except Exception as e:
    print(str(e))


In [ ]:
import os
from Auth.MsalWindowsAuthenticator import MsalWindowsAuthenticator

CLIENT_ID = os.environ.get('CLIENT_ID')
TENANT_ID = os.environ.get('TENANT_ID')
SCOPES = [os.environ.get('SCOPE_USER')]

winauth = MsalWindowsAuthenticator(CLIENT_ID, TENANT_ID, SCOPES)

try:
    wtoken = winauth.GetAuthtoken()
    print("Token acquired successfully")
except Exception as e:
    print(str(e))

In [ ]:
from Client.OriginClient import OriginClient

oclient = OriginClient("http://localhost:4002/graphql", "ws://localhost:4002/graphql", winauth)

query = """
    query getProjects {
      projects {
        Name
      }
    }
"""

qresult = await oclient.QueryAsync(query)
print(qresult)


In [ ]:
import asyncio
from asyncio import TaskGroup
from DesignBehaviours.SpaceOutletsBehaviour import SpaceOutletsBehaviour

spaceTypeBehaviour1 = SpaceOutletsBehaviour(oclient)
subresult1 = spaceTypeBehaviour1.Start()

oclient2 = OriginClient("http://localhost:4002/graphql", "ws://localhost:4002/graphql", winauth)

spaceTypeBehaviour2 = SpaceOutletsBehaviour(oclient2)
subresult2 = spaceTypeBehaviour2.Start()

try:
    async with TaskGroup() as group:
                # spawn some tasks
                group.create_task(subresult1)
                #group.create_task(subresult2)

                await asyncio.sleep(1000)
                raise Exception("Tasks timed out")
except Exception as e:
        print(e)
        pass

await oclient.Close()
await oclient2.Close()

In [ ]:
from Client.OriginQueries import Query

projectID = await Query.GetProjectIdForSpaceId("<add a space id here>", oclient)

print(projectID)

entityTypes = await Query.GetSocketTypesForProject(projectID, oclient)

print(entityTypes)

def get_id_by_name(entity_types, name):
    for entity in entity_types:
        if entity['Name'] == name:
            return entity['Id']
    return None 

typeId_230vsocket =  get_id_by_name(entityTypes, "230v Single Socket Non Essential")

print(typeId_230vsocket)